In [18]:
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import argparse
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [19]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=196)

In [20]:
def getImageAlgin(imgPath):
    image = cv2.imread(imgPath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)
    if len(faces) > 0:
        rect = faces[0]
        faceAligned = fa.align(image, gray, rect)
        rects = detector(faceAligned, 1)
        if len(rects) > 0:
            rect = detector(faceAligned, 1)[0]
            (x, y, w, h) = rect_to_bb(rect)
            return cv2.resize(faceAligned[y:y + h, x:x + w], (96,96))
        else :
            return False
    else :
        return False

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
from utils import LRN2D
import utils
import numpy as np


In [4]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)


InternalError: Failed to create session.

In [5]:
from keras.models import load_model
# model = load_model('./model/nn4.small2.lrn.h5')
from keras.utils import CustomObjectScope
import tensorflow as tf
with CustomObjectScope({'tf': tf}):
    model = load_model('./model/nn4.small2.lrn.h5')



/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
def get128keypoint(imgPath):
    faceAligned = getImageAlgin(imgPath)
    if faceAligned is not False:
        img = faceAligned[...,::-1]
        img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
        x_train = np.array([img])
        y1 = model.predict_on_batch(x_train)
        return y1[0]
    else :
        return False

In [21]:
get128keypoint('./images/mm7.jpg')

error: OpenCV(3.4.3) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [39]:
y1 = get128keypoint('./images/pp9.jpg')
y2 = get128keypoint('./images/thin8.jpg')
dist = np.sqrt(np.sum(np.square(np.subtract(y1[0], y2[0]))))
dist


0.05377739

In [22]:
persons = glob.glob("./Enouvo_company/*")
index = -1
dataTrain = []
labelTrain = []

dataTest = []
labelTest = []
for person in persons:
    print(person)
    index = index + 1
    imagesTrain = glob.glob(person + "/Train/*.*")
    for image in imagesTrain:
        k = get128keypoint(image)
        if k is not False:
            dataTrain.append(k)
            labelTrain.append(index)
    imagesTest = glob.glob(person + "/Test/*.*")
    for image in imagesTest:
        dataTest.append(image)
        labelTest.append(index)
    

./Enouvo_company/BuiMinhPhong
./Enouvo_company/DangHy
./Enouvo_company/HoThiKhanhLy
./Enouvo_company/JasonNguyen
./Enouvo_company/MaiThiThuThuy
./Enouvo_company/NguyenHuuTuongVi
./Enouvo_company/NguyenLeBaoTram
./Enouvo_company/NguyenLinhGiang
./Enouvo_company/NguyenNhatNam
./Enouvo_company/NguyenThiThanhPhuong
./Enouvo_company/NguyenTranAnhPhuong
./Enouvo_company/NguyenTriCong
./Enouvo_company/NguyenXuanHung
./Enouvo_company/PhamSiNguyen
./Enouvo_company/PhamThiNgocAnh
./Enouvo_company/TranHanhTrang
./Enouvo_company/TranTHiMyLinh
./Enouvo_company/TranThiThuHien
./Enouvo_company/TruongThiYNhi
./Enouvo_company/VoThiThanhTuyen


In [32]:
print(labelTrain)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [7]:
data = []
label = []
listfolder = glob.glob("./images/train/*")
for i, value in enumerate(listfolder):
    listfile = glob.glob(value + "/*.*")
    for img in listfile:
        print(img)
        data.append(get128keypoint(img))
        label.append(i)



./images/train/mm/mm.jpg
./images/train/mm/mm10.jpg
./images/train/mm/mm11.jpg
./images/train/mm/mm12.jpg
./images/train/mm/mm2.jpg
./images/train/mm/mm3.jpg
./images/train/mm/mm5.jpg
./images/train/mm/mm6.jpg
./images/train/mm/mm7.jpg
./images/train/mm/mm8.jpg
./images/train/mm/mm9.jpg
./images/train/pp/pp.jpg
./images/train/pp/pp10.jpg
./images/train/pp/pp2.jpg
./images/train/pp/pp3.jpg
./images/train/pp/pp4.jpg
./images/train/pp/pp5.jpg
./images/train/pp/pp6.jpg
./images/train/pp/pp7.jpg
./images/train/pp/pp8.jpg
./images/train/quynh/quynh1.jpg
./images/train/quynh/quynh2.jpg
./images/train/quynh/quynh4.jpg
./images/train/quynh/quynh5.jpg
./images/train/quynh/quynh6.jpg
./images/train/quynh/quynh7.jpg
./images/train/quynh/quynh8.jpg
./images/train/quynh/quynh9.jpg
./images/train/thien/thien.jpg
./images/train/thien/thien1.jpg
./images/train/thien/thien2.jpg
./images/train/thien/thien3.jpg
./images/train/thien/thien4.jpg
./images/train/thien/thien5.jpg
./images/train/thien/thien6.jpg

In [8]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(dataTrain, labelTrain)

GaussianNB(priors=None, var_smoothing=1e-09)

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = [
            {'C': [1, 10, 100, 1000],
             'kernel': ['linear']},
            {'C': [1, 10, 100, 1000],
             'gamma': [0.001, 0.0001],
             'kernel': ['rbf']}
        ]
clf = GridSearchCV(SVC(C=1, probability=True), param_grid, cv=5)
clf.fit(dataTrain, labelTrain)   

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
result = 0
for idx, testImg in enumerate(dataTest):
    img = get128keypoint(testImg)
    if img is not False:
        rep = img.reshape(1, -1)
        predictions = clf.predict_proba(rep).ravel()
        maxI = np.argmax(predictions)
        if (maxI == labelTest[idx] ):
            result = result + 1
            print(testImg)
        else :
            print(maxI)
    else : 
        print("wrong")

./Enouvo_company/BuiMinhPhong/Test/56367274_1866853146752034_1279803514364624896_n.jpg
./Enouvo_company/BuiMinhPhong/Test/Screenshot_6.png
16
./Enouvo_company/DangHy/Test/Screenshot_1.png
6
./Enouvo_company/DangHy/Test/Screenshot_8.png
16
./Enouvo_company/HoThiKhanhLy/Test/Screenshot_1.png
./Enouvo_company/HoThiKhanhLy/Test/Screenshot_3.png
./Enouvo_company/JasonNguyen/Test/Screenshot_1.png
0
./Enouvo_company/JasonNguyen/Test/Screenshot_3.png
./Enouvo_company/MaiThiThuThuy/Test/Screenshot_1.png
16
./Enouvo_company/MaiThiThuThuy/Test/Screenshot_8.png
./Enouvo_company/NguyenHuuTuongVi/Test/Screenshot_1.png
./Enouvo_company/NguyenHuuTuongVi/Test/Screenshot_2.png
./Enouvo_company/NguyenHuuTuongVi/Test/Screenshot_3.png
9
./Enouvo_company/NguyenLeBaoTram/Test/Screenshot_7.png
./Enouvo_company/NguyenLeBaoTram/Test/Screenshot_8.png
9
wrong
6
14
9
./Enouvo_company/NguyenNhatNam/Test/Screenshot_2.png
./Enouvo_company/NguyenNhatNam/Test/Screenshot_3.png
./Enouvo_company/NguyenThiThanhPhuong/Test/

In [25]:
print(result)
print(len(dataTest))

37
63


In [19]:
rep = get128keypoint('./images/xp6.jpg').reshape(1, -1)
predictions = clf.predict_proba(rep).ravel()
maxI = np.argmax(predictions)
confidence = predictions[maxI]
print(predictions, confidence)

[2.06596053e-17 3.92894114e-11 6.83670456e-35 1.00000000e+00
 1.32545410e-12 4.48815417e-70] 0.9999999999593854


In [43]:
def get128keypointTest(pathname):
    img = cv2.imread(pathname)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR)
    
    if img is not False:
        img = img[...,::-1]
        img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
        x_train = np.array([img])
        y1 = model.predict_on_batch(x_train)
        return y1[0]
    else :
        return False

In [44]:
get128keypointTest("./images/pp9.jpg")

array([ 0.0487835 ,  0.02638223, -0.07230595, -0.07985031,  0.0052608 ,
        0.08356217, -0.01922345, -0.01764362,  0.02354503, -0.07758575,
       -0.01393996,  0.07211049,  0.10336382, -0.114964  , -0.08214266,
       -0.0149932 , -0.15426385, -0.09215006, -0.16866918,  0.2300697 ,
        0.07824177, -0.02291875,  0.09083769,  0.05731176, -0.16006744,
        0.08460534, -0.05663537, -0.17049293, -0.01825422,  0.03342596,
       -0.05120825,  0.04528322, -0.01049708,  0.10250849, -0.02599949,
        0.03421171,  0.0191655 ,  0.28396183, -0.08411465, -0.00988444,
       -0.04047473, -0.06698567, -0.08416975,  0.04996787, -0.0839885 ,
        0.00601189,  0.12507719,  0.06885215, -0.07777947,  0.21923834,
       -0.04018158, -0.00929337, -0.06666382,  0.12204283,  0.20658225,
       -0.08266607,  0.01025563,  0.04867583, -0.05833682, -0.17933074,
       -0.04935848, -0.02176182,  0.20164338, -0.03364203,  0.00150317,
       -0.14073004,  0.0474381 ,  0.0888806 , -0.00668177,  0.00